# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
#importing the SparkSession
from pyspark.sql import SparkSession

#import pandas
import pandas as pd

#initiate spark
spark = SparkSession.builder.appName('hackers').getOrCreate()

#loading the dataset
raw_data = spark.read.csv('15.hack_data.csv', inferSchema=True, header=True)

### Exploratory analysis and Data Preprocessing

In [2]:
#explore the data in pandas for better visualization
pd.DataFrame(raw_data.collect(), columns=raw_data.columns)

,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed
0,8,391.09,1,2.96,7,Slovenia,72.37
1,20,720.99,0,3.04,9,British Virgin Islands,69.08
2,31,356.32,1,3.71,8,Tokelau,70.58
3,2,228.08,1,2.48,8,Bolivia,70.80
4,20,408.50,0,3.57,8,Iraq,71.28
...,...,...,...,...,...,...,...
329,39,761.91,1,6.99,14,Belarus,43.23
330,43,983.48,0,8.60,13,British Indian Ocean Territory (Chagos Archipe...,43.21
331,39,690.22,1,6.80,13,Guinea-Bissau,42.75
332,36,1060.69,1,6.26,14,Canada,43.86


In [3]:
#explore data stats
summary_df = pd.DataFrame(raw_data.describe().collect(), 
             columns=raw_data.describe().columns
            ).set_index('summary')
summary_df

,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed
summary,,,,,,,
count,334,333,334,333,334,332,334
mean,30.008982035928145,608.026096096096,0.5119760479041916,5.2654054054054065,10.838323353293413,None,57.342395209580864
stddev,14.088200614636158,286.41048942415415,0.5006065264451406,2.3019067988197586,3.06352633036022,None,13.41106336843464
min,1,10.0,0,1.0,6,Afghanistan,40.0
max,60,1330.5,1,10.0,15,Zimbabwe,75.0


In [4]:
#explore the columns
raw_data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [5]:
#explore the schema
raw_data.printSchema()

root
 |-- Session_Connection_Time: integer (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [6]:
#check if there are any missing values
from pyspark.sql.functions import (isnan, isnull, count, countDistinct)

In [7]:
#remove missing values
data_no_mv = raw_data.na.drop()

In [8]:
#check the stats
pd.DataFrame(data_no_mv.describe().collect(), 
             columns=data_no_mv.describe().columns
            ).set_index('summary')

,Session_Connection_Time,Bytes Transferred,Kali_Trace_Used,Servers_Corrupted,Pages_Corrupted,Location,WPM_Typing_Speed
summary,,,,,,,
count,330,330,330,330,330,330,330
mean,30.124242424242425,608.6308787878787,0.5060606060606061,5.289272727272728,10.881818181818181,None,57.18948484848488
stddev,14.121262295433512,287.05347758043047,0.5007225138032231,2.298289059314732,3.0559080797310796,None,13.418058354596946
min,1,10.0,0,1.0,6,Afghanistan,40.0
max,60,1330.5,1,10.0,15,Zimbabwe,75.0


### Assemble the input vector

In [9]:
#import the vector assembler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

#create a Vector Assembler instance
assembler = VectorAssembler(inputCols=['Session_Connection_Time','Bytes Transferred','Kali_Trace_Used',
                                  'Servers_Corrupted','Pages_Corrupted','WPM_Typing_Speed'], 
                           outputCol='features')
#apply the assembler to the dataset
data_no_mv_assembled = assembler.transform(data_no_mv)

### Scale the data

In [10]:
#import the standard scaler
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')

#apply the scaler to the dataset
data_scaled = scaler.fit(data_no_mv_assembled).transform(data_no_mv_assembled)

#comparing the scaled and non-scaled features
data_scaled.select('features', 'features_scaled').show()

+--------------------+--------------------+
|            features|     features_scaled|
+--------------------+--------------------+
|[8.0,391.09,1.0,2...|[0.56652159223662...|
|[20.0,720.99,0.0,...|[1.41630398059156...|
|[31.0,356.32,1.0,...|[2.19527116991691...|
|[2.0,228.08,1.0,2...|[0.14163039805915...|
|[20.0,408.5,0.0,3...|[1.41630398059156...|
|[1.0,390.69,1.0,2...|[0.07081519902957...|
|[18.0,342.97,1.0,...|[1.27467358253240...|
|[22.0,101.61,1.0,...|[1.55793437865071...|
|[15.0,275.53,1.0,...|[1.06222798544367...|
|[12.0,424.83,1.0,...|[0.84978238835493...|
|[15.0,249.09,1.0,...|[1.06222798544367...|
|[32.0,242.48,0.0,...|[2.26608636894649...|
|[23.0,514.54,0.0,...|[1.62874957768029...|
|[9.0,284.77,0.0,3...|[0.63733679126620...|
|[21.0,355.94,1.0,...|[1.48711917962113...|
|[10.0,372.65,0.0,...|[0.70815199029578...|
|[22.0,456.57,0.0,...|[1.55793437865071...|
|[25.0,582.03,0.0,...|[1.77037997573945...|
|[19.0,67.17,0.0,3...|[1.34548878156198...|
|[16.0,410.08,1.0,...|[1.1330431

### Model

#### There are no labels. We will use unsupervised learning. There will be either two or three clusters. After testing each k value (2 or 3), we will check the number of observations for each cluster. The one with even distribution of observation will be our pick. 
####  if (110x3) then k=3,  if (165x2) then k=2


In [11]:
#import the KMeans 
from pyspark.ml.clustering import KMeans

for k in range(2,4):
    
    #create a KMeans instance
    kmeans = KMeans(featuresCol='features_scaled', predictionCol= f'prediction with {k} clusters', k=k )
    
    #train the model
    model = kmeans.fit(data_scaled)
    
    #apply the model to the data    
    result = model.transform(data_scaled)
    
    #evaluate WCSSE
    wcsse = model.computeCost(data_scaled)
    print(f'The WCSSE for {k} clusters is {wcsse}')
    
    #count the number of observations for each cluster
    print(f'The number of observations for each cluster:')
    result.groupby(f'prediction with {k} clusters').count().show()
    

The WCSSE for 2 clusters is 593.0557710463897
The number of observations for each cluster:
+--------------------------+-----+
|prediction with 2 clusters|count|
+--------------------------+-----+
|                         1|  167|
|                         0|  163|
+--------------------------+-----+

The WCSSE for 3 clusters is 426.12076755773785
The number of observations for each cluster:
+--------------------------+-----+
|prediction with 3 clusters|count|
+--------------------------+-----+
|                         1|   79|
|                         2|   88|
|                         0|  163|
+--------------------------+-----+



## The Two-cluster option is the right choice since all observations/attacks are distributed evenly across each hacker.